In [ ]:
import random
import re
from pathlib import Path
from sklearn.model_selection import train_test_split

## Data Science
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## torch
import torch
from torch import nn
from torch.nn import functional as F
from torch import optim
from torch.utils.data import Dataset, DataLoader

In [ ]:
from sklearn.datasets import load_digits
digits = load_digits()

In [ ]:
features, targets = digits.data, digits.target
features.shape, targets.shape

((1797, 64), (1797,))

In [ ]:
img = features[32].reshape(8,8).astype(int)
img

array([[ 0,  2, 13, 16, 16, 16, 11,  0],
       [ 0,  5, 16, 10,  5,  4,  1,  0],
       [ 0,  6, 16,  7,  3,  0,  0,  0],
       [ 0,  9, 16, 16, 16,  6,  0,  0],
       [ 0,  3,  8,  4, 11, 15,  0,  0],
       [ 0,  0,  0,  1, 12, 15,  0,  0],
       [ 0,  0,  4, 13, 16,  6,  0,  0],
       [ 0,  2, 16, 15,  8,  0,  0,  0]])

In [ ]:
import pandas as pd
df = pd.DataFrame(img)
df

,0,1,2,3,4,5,6,7
0,0,2,13,16,16,16,11,0
1,0,5,16,10,5,4,1,0
2,0,6,16,7,3,0,0,0
3,0,9,16,16,16,6,0,0
4,0,3,8,4,11,15,0,0
5,0,0,0,1,12,15,0,0
6,0,0,4,13,16,6,0,0
7,0,2,16,15,8,0,0,0


In [ ]:
df.style.set_properties(**{'font-size':'8pt'}).background_gradient('Greys')

,0,1,2,3,4,5,6,7
0,0,2,13,16,16,16,11,0
1,0,5,16,10,5,4,1,0
2,0,6,16,7,3,0,0,0
3,0,9,16,16,16,6,0,0
4,0,3,8,4,11,15,0,0
5,0,0,0,1,12,15,0,0
6,0,0,4,13,16,6,0,0
7,0,2,16,15,8,0,0,0


In [ ]:
targets[32]

5

In [ ]:
features = features/255

In [ ]:
features

array([[0.        , 0.        , 0.01960784, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.03921569, 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.0627451 , 0.03529412,
        0.        ],
       ...,
       [0.        , 0.        , 0.00392157, ..., 0.02352941, 0.        ,
        0.        ],
       [0.        , 0.        , 0.00784314, ..., 0.04705882, 0.        ,
        0.        ],
       [0.        , 0.        , 0.03921569, ..., 0.04705882, 0.00392157,
        0.        ]])

In [ ]:
class TabDataSet(Dataset):
  def __init__(self, features, targets):
    self.x = features
    self.y  = targets
  def __len__(self): return len(self.x)
  def __getitem__(self, idx): return self.x[idx], self.y[idx]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(features, targets, test_size=0.2, random_state = 42)

In [ ]:
batch = 128

In [ ]:
##DataSet
train_ds = TabDataSet(X_train, y_train)
train_ds[0]

(array([0.        , 0.        , 0.01176471, 0.05490196, 0.00392157,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.04705882, 0.04705882, 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.01176471, 0.0627451 , 0.02352941,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.01960784, 0.0627451 , 0.00784314, 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.02352941, 0.0627451 ,
        0.00784314, 0.01960784, 0.00784314, 0.        , 0.        ,
        0.        , 0.01568627, 0.0627451 , 0.00784314, 0.04705882,
        0.05882353, 0.00784314, 0.        , 0.        , 0.00392157,
        0.05490196, 0.05098039, 0.00784314, 0.05098039, 0.04313725,
        0.        , 0.        , 0.        , 0.01176471, 0.04313725,
        0.0627451 , 0.05098039, 0.01568627, 0.        ]), 6)

In [ ]:
test_ds = TabDataSet(X_test, y_test)
test_ds[0]

(array([0.        , 0.        , 0.        , 0.02745098, 0.04705882,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.01568627, 0.0627451 , 0.03137255, 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.04705882, 0.04313725,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.05882353, 0.03921569, 0.03137255, 0.02352941,
        0.00392157, 0.        , 0.        , 0.        , 0.05882353,
        0.0627451 , 0.03137255, 0.03921569, 0.03137255, 0.        ,
        0.        , 0.        , 0.05490196, 0.02745098, 0.        ,
        0.        , 0.04705882, 0.        , 0.        , 0.        ,
        0.03137255, 0.04313725, 0.        , 0.01960784, 0.0627451 ,
        0.00784314, 0.        , 0.        , 0.        , 0.03529412,
        0.05490196, 0.05490196, 0.01960784, 0.        ]), 6)

In [ ]:
train_dl = DataLoader(train_ds, batch_size=batch, shuffle = True)
test_dl = DataLoader(test_ds, batch_size=batch, shuffle=True)

In [ ]:
def batch_accuracy(xb, yb):
  preds = torch.sigmoid(xb)
  correct = (preds.argmax(1))  == yb # Taking the index of max value  i.e. max probaabilty
  return correct.float().mean()

In [ ]:
def validate_epoch(model, metric):
  accs = [metric(model(xb.float()), yb) for xb,yb in test_dl]
  return round(torch.stack(accs).mean().item(), 4)

In [ ]:
def train_model(model, epochs, metric = batch_accuracy):
  for i  in range(epochs):
    for xb, yb in train_dl:
      preds = model(xb.float())
      loss = loss_func(preds, yb)
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
      
    print(validate_epoch(model, metric), end=" ")

In [ ]:
def loss_func(inp, tar):
  loss =  nn.CrossEntropyLoss()
  return loss(inp, tar.long())

In [ ]:
def get_model(n_in, n_out):
  return nn.Sequential(nn.Linear(n_in, 128), 
                       nn.ReLU(),
                       nn.Linear(128, 64),
                       nn.ReLU(),
                       nn.Linear(64, 32),
                       nn.ReLU(),
                       nn.Linear(32,n_out))

In [ ]:
model = get_model(64,10)
optimizer  = torch.optim.Adam(model.parameters(), lr=0.01)

In [ ]:
#train_model(model, 5)

In [ ]:
train_model(model, 5)

0.152 0.4926 0.6326 0.651 0.7714 

In [ ]:
!pip install aiadv

In [ ]:
from aiadv import URLs